## Experiments on _Rfam novel_ datasets

In [1]:
import numpy
import sklearn
import tensorflow
import matplotlib
import pandas

print(tensorflow.__version__)
print(sklearn.__version__)
print(numpy.__version__)
print(matplotlib.__version__)
print(pandas.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1
0.21.3
1.18.5
3.1.1
0.24.2


In [2]:
import numpy as np
from tensorflow import keras
from sklearn import preprocessing

# Load labels 

train_labels=np.load("train_labels.npy")
val_labels=np.load("val_labels.npy")
test_labels=np.load("test_labels.npy")

num_classes = len(np.unique(train_labels))
print('Total classes: ',num_classes)

# Encodes labels to categorical

le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels_num = le.transform(train_labels)
val_labels_num = le.transform(val_labels)
test_labels_num = le.transform(test_labels)

train_labels_bin = keras.utils.to_categorical(train_labels_num, num_classes)
val_labels_bin = keras.utils.to_categorical(val_labels_num, num_classes)
test_labels_bin = keras.utils.to_categorical(test_labels_num, num_classes)




Total classes:  88


In [ ]:
from utils.ExpConfiguration import *
from utils.modelUtils import *
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)


padd = 'random' # CHANGE HERE to select other padding schemas (new, constant, random)

batch_size = 32
epochs = 20

# where results are stored
v_acc = {}

for nl in nlayers:
    venc = {}
    for en in seqEncoders:
        vbnoise = {}
        for bn in bnoise:
            print('n. layers=',nl,' Encoder=',en['filename'],' Noise=',str(bn), ' Padding=',padd)
            train_seq=np.load('train_' + en['filename'] + '_' + padd + '_' + str(bn) + '_seq.npy')
            val_seq=np.load('val_' + en['filename'] + '_' + padd + '_' + str(bn) + '_seq.npy')
            test_seq=np.load('test_' + en['filename'] + '_' + padd + '_' + str(bn) + '_seq.npy')

            if (nl>0):
                train_seq = keras.utils.to_categorical(train_seq)
                val_seq = keras.utils.to_categorical(val_seq)
                test_seq = keras.utils.to_categorical(test_seq)
            else:
                train_seq = train_seq.reshape((-1,np.prod(train_seq.shape[1:])))
                val_seq = val_seq.reshape((-1,np.prod(val_seq.shape[1:])))
                test_seq = test_seq.reshape((-1,np.prod(test_seq.shape[1:])))
                nmax = np.max(train_seq)
                train_seq = train_seq/nmax
                val_seq = val_seq/nmax
                test_seq = test_seq/nmax

            tf.keras.backend.clear_session()
            print(train_seq.shape)
            print(val_seq.shape)
            print(test_seq.shape)
            if (en['filename'] in ['1mer','2mer','3mer']):
                m=buildCNNModel(inshape=train_seq.shape[1:],num_classes=num_classes,nlayers=nl,cnndim=1)
            else:
                m=buildCNNModel(inshape=train_seq.shape[1:],num_classes=num_classes,nlayers=nl,cnndim=2)
            print(m.summary())
            m.compile(optimizer=keras.optimizers.Adam(lr=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
            m.fit(train_seq, train_labels_bin,verbose=1,
                  batch_size=batch_size,shuffle=True,
                  epochs=epochs,#validation_split=0.33,
                  validation_data=(val_seq, val_labels_bin))

            pred = m.predict(test_seq, verbose=1)
            predicted = np.argmax(pred, axis=1)
            
            vbnoise.update({str(bn) : predicted})
        venc.update({en['filename'] : vbnoise})
    v_acc.update({str(nl) : venc})




n. layers= 0  Encoder= 3mer  Noise= 0  Padding= random
(105864, 67)
(17324, 67)
(25342, 67)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               34000     
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              501000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               100100    
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 88)                8888  

105864/105864 [==============================] - 30s 285us/sample - loss: 0.7344 - acc: 0.8106 - val_loss: 2.8816 - val_acc: 0.4116
Epoch 16/20
105864/105864 [==============================] - 30s 284us/sample - loss: 0.7155 - acc: 0.8143 - val_loss: 3.1701 - val_acc: 0.3911
Epoch 17/20
105864/105864 [==============================] - 30s 283us/sample - loss: 0.6968 - acc: 0.8196 - val_loss: 3.0906 - val_acc: 0.4048
Epoch 18/20
105864/105864 [==============================] - 30s 283us/sample - loss: 0.6868 - acc: 0.8221 - val_loss: 3.0292 - val_acc: 0.4205
Epoch 19/20
105864/105864 [==============================] - 30s 284us/sample - loss: 0.6714 - acc: 0.8255 - val_loss: 3.1233 - val_acc: 0.4065
Epoch 20/20
25342/25342 [==============================] - 1s 42us/sample
n. layers= 0  Encoder= 3mer  Noise= 50  Padding= random
(105864, 100)
(17324, 100)
(25342, 100)
_________________________________________________________________
Layer (type)                 Output Shape              P

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



105864/105864 [==============================] - 29s 270us/sample - loss: 0.8869 - acc: 0.7690 - val_loss: 3.0166 - val_acc: 0.3762
Epoch 19/20
105864/105864 [==============================] - 28s 267us/sample - loss: 0.8702 - acc: 0.7747 - val_loss: 3.1664 - val_acc: 0.3906
Epoch 20/20
25342/25342 [==============================] - 1s 42us/sample
n. layers= 0  Encoder= 3mer  Noise= 75  Padding= random
(105864, 117)
(17324, 117)
(25342, 117)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               59000     
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              501000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               100100    
_________________________________________________________________
dropout (Dropout)         

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

105864/105864 [==============================] - 30s 283us/sample - loss: 1.0548 - acc: 0.7412 - val_loss: 3.7682 - val_acc: 0.2761
Epoch 19/20
105864/105864 [==============================] - 30s 282us/sample - loss: 1.0106 - acc: 0.7500 - val_loss: 4.1701 - val_acc: 0.2591
Epoch 20/20
25342/25342 [==============================] - 1s 48us/sample
n. layers= 0  Encoder= Morton  Noise= 75  Padding= random
(105864, 1024)
(17324, 1024)
(25342, 1024)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               512500    
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              501000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               100100    
_________________________________________________________________
dropout (Dropout)    

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



105864/105864 [==============================] - 30s 282us/sample - loss: 2.0531 - acc: 0.5218 - val_loss: 3.9195 - val_acc: 0.1470
Epoch 7/20
105864/105864 [==============================] - 30s 280us/sample - loss: 1.9495 - acc: 0.5424 - val_loss: 3.7408 - val_acc: 0.2023
Epoch 8/20
 69056/105864 [==================>...........] - ETA: 9s - loss: 1.8457 - acc: 0.5646

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 12288/105864 [==>...........................] - ETA: 23s - loss: 1.7142 - acc: 0.5886

In [ ]:
import pickle

# save results on file
f = open('results/Rfam2Novel_' + padd + '.pckl', 'wb')
pickle.dump(v_acc, f)
f.close()

